Module imports

In [1]:
import geopandas as gpd
import fiona

# HEX 5

Hex imports

In [2]:
%%time
# Define the path to the geodatabase
inputs_gdb = r'C://Research/Grid_effort/H3Grid_Inputs.gdb'

hexLevel = '5'

# Read in the H3_5 polygon grid layer as the base layer
base_layer = gpd.read_file(inputs_gdb, layer='H3_'+hexLevel)
base_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)

CPU times: total: 156 ms
Wall time: 148 ms


Hex to counties

In [3]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='tj_2021_us_st_cnt')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 1.52 s
Wall time: 1.51 s


In [4]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 93.8 ms
Wall time: 94.6 ms


In [5]:
%%time
output_layer.to_file('h'+hexLevel+'_tj2021.gdb',driver='OpenFileGDB')

CPU times: total: 203 ms
Wall time: 212 ms


In [6]:
del joined_layer, output_layer

Hex to estuarine

In [7]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='Estuarine_Drainage_Areas')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 203 ms
Wall time: 207 ms


In [8]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 46.9 ms
Wall time: 47.3 ms


In [9]:
%%time
output_layer.to_file('h'+hexLevel+'_Estuarine.gdb',driver='OpenFileGDB')

CPU times: total: 109 ms
Wall time: 139 ms


In [10]:
del joined_layer, output_layer

Hex to huc8

In [11]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='WBDHU8')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 23.5 s
Wall time: 23.5 s


In [12]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 422 ms
Wall time: 419 ms


In [13]:
%%time
output_layer.to_file('h'+hexLevel+'_WBDHU8.gdb',driver='OpenFileGDB')

CPU times: total: 172 ms
Wall time: 265 ms


In [14]:
del joined_layer, output_layer

Hex to census

In [15]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='dtl_cnty_Census_ESRI')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 8.58 s
Wall time: 8.6 s


In [16]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 391 ms
Wall time: 389 ms


In [17]:
%%time
output_layer.to_file('h'+hexLevel+'_Census.gdb',driver='OpenFileGDB')

CPU times: total: 141 ms
Wall time: 169 ms


In [18]:
del joined_layer, output_layer

Hex to huc12

In [19]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='WBDHU12')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 8.52 s
Wall time: 8.53 s


In [20]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 250 ms
Wall time: 254 ms


In [21]:
%%time
output_layer.to_file('h'+hexLevel+'_WBDHU12.gdb',driver='OpenFileGDB')

CPU times: total: 375 ms
Wall time: 393 ms


In [22]:
del joined_layer, output_layer

Hex to all at once

In [23]:
%%time
layers_to_join = ['tj_2021_us_st_cnt', 'Estuarine_Drainage_Areas', 'WBDHU8', 'dtl_cnty_Census_ESRI', 'WBDHU12']
layer_gdfs = {layer: gpd.read_file(inputs_gdb, layer=layer) for layer in layers_to_join}

CPU times: total: 43.1 s
Wall time: 43.2 s


In [24]:
%%time
for name, gdf in layer_gdfs.items():
    # Ensure the CRS is the same between the base layer and the current layer
    if not gdf.crs == base_layer.crs:
        gdf = gdf.to_crs(base_layer.crs)
    if 'Shape_Length' in gdf.columns:
        gdf.drop('Shape_Length', axis=1, inplace=True)
    if 'Shape_Area' in gdf.columns:
        gdf.drop('Shape_Area', axis=1, inplace=True)

    # Perform spatial join (inner join by default)
    base_layer = gpd.sjoin(base_layer, gdf, how="left")
    
    # Drop the `index_right` column if it exists
    if 'index_right' in base_layer.columns:
        base_layer.drop('index_right', axis=1, inplace=True)

CPU times: total: 4.92 s
Wall time: 4.92 s


In [25]:
%%time
base_layer.to_file('h'+hexLevel+'_allAtOnce.gdb',driver='OpenFileGDB')

CPU times: total: 5.05 s
Wall time: 5.07 s


In [26]:
del layers_to_join, layer_gdfs, gdf, base_layer

# HEX 6

Hex imports

In [27]:
%%time
# Define the path to the geodatabase
inputs_gdb = r'C://Research/Grid_effort/H3Grid_Inputs.gdb'

hexLevel = '6'

# Read in the H3_5 polygon grid layer as the base layer
base_layer = gpd.read_file(inputs_gdb, layer='H3_'+hexLevel)
base_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)

CPU times: total: 719 ms
Wall time: 717 ms


Hex to counties

In [28]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='tj_2021_us_st_cnt')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 1.45 s
Wall time: 1.44 s


In [29]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 359 ms
Wall time: 342 ms


In [30]:
%%time
output_layer.to_file('h'+hexLevel+'_tj2021.gdb',driver='OpenFileGDB')

CPU times: total: 734 ms
Wall time: 748 ms


In [31]:
del joined_layer, output_layer

Hex to estuarine

In [32]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='Estuarine_Drainage_Areas')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 203 ms
Wall time: 211 ms


In [33]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 219 ms
Wall time: 221 ms


In [34]:
%%time
output_layer.to_file('h'+hexLevel+'_Estuarine.gdb',driver='OpenFileGDB')

CPU times: total: 656 ms
Wall time: 647 ms


In [35]:
del joined_layer, output_layer

Hex to huc8

In [36]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='WBDHU8')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 23.8 s
Wall time: 23.8 s


In [37]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 2.05 s
Wall time: 2.05 s


In [38]:
%%time
output_layer.to_file('h'+hexLevel+'_WBDHU8.gdb',driver='OpenFileGDB')

CPU times: total: 703 ms
Wall time: 709 ms


In [39]:
del joined_layer, output_layer

Hex to census

In [40]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='dtl_cnty_Census_ESRI')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 8.73 s
Wall time: 8.75 s


In [41]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 2.22 s
Wall time: 2.22 s


In [42]:
%%time
output_layer.to_file('h'+hexLevel+'_Census.gdb',driver='OpenFileGDB')

CPU times: total: 672 ms
Wall time: 676 ms


In [43]:
del joined_layer, output_layer

Hex to huc12

In [44]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='WBDHU12')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 8.59 s
Wall time: 8.63 s


In [45]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 656 ms
Wall time: 642 ms


In [46]:
%%time
output_layer.to_file('h'+hexLevel+'_WBDHU12.gdb',driver='OpenFileGDB')

CPU times: total: 1.11 s
Wall time: 1.11 s


In [47]:
del joined_layer, output_layer

Hex to all at once

In [48]:
%%time
layers_to_join = ['tj_2021_us_st_cnt', 'Estuarine_Drainage_Areas', 'WBDHU8', 'dtl_cnty_Census_ESRI', 'WBDHU12']
layer_gdfs = {layer: gpd.read_file(inputs_gdb, layer=layer) for layer in layers_to_join}

CPU times: total: 43.2 s
Wall time: 43.2 s


In [49]:
%%time
for name, gdf in layer_gdfs.items():
    # Ensure the CRS is the same between the base layer and the current layer
    if not gdf.crs == base_layer.crs:
        gdf = gdf.to_crs(base_layer.crs)
    if 'Shape_Length' in gdf.columns:
        gdf.drop('Shape_Length', axis=1, inplace=True)
    if 'Shape_Area' in gdf.columns:
        gdf.drop('Shape_Area', axis=1, inplace=True)

    # Perform spatial join (inner join by default)
    base_layer = gpd.sjoin(base_layer, gdf, how="left")
    
    # Drop the `index_right` column if it exists
    if 'index_right' in base_layer.columns:
        base_layer.drop('index_right', axis=1, inplace=True)

CPU times: total: 10.2 s
Wall time: 10.2 s


In [50]:
%%time
base_layer.to_file('h'+hexLevel+'_allAtOnce.gdb',driver='OpenFileGDB')

CPU times: total: 4.31 s
Wall time: 4.32 s


In [51]:
del layers_to_join, layer_gdfs, gdf, base_layer

# HEX 7

Hex imports

In [52]:
%%time
# Define the path to the geodatabase
inputs_gdb = r'C://Research/Grid_effort/H3Grid_Inputs.gdb'

hexLevel = '7'

# Read in the H3_5 polygon grid layer as the base layer
base_layer = gpd.read_file(inputs_gdb, layer='H3_'+hexLevel)
base_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)

CPU times: total: 4.69 s
Wall time: 4.68 s


Hex to counties

In [53]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='tj_2021_us_st_cnt')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 1.55 s
Wall time: 1.55 s


In [54]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 2.06 s
Wall time: 2.06 s


In [55]:
%%time
output_layer.to_file('h'+hexLevel+'_tj2021.gdb',driver='OpenFileGDB')

CPU times: total: 4.47 s
Wall time: 4.49 s


In [56]:
del joined_layer, output_layer

Hex to estuarine

In [57]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='Estuarine_Drainage_Areas')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 234 ms
Wall time: 240 ms


In [58]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 1.5 s
Wall time: 1.48 s


In [59]:
%%time
output_layer.to_file('h'+hexLevel+'_Estuarine.gdb',driver='OpenFileGDB')

CPU times: total: 3.91 s
Wall time: 3.92 s


In [60]:
del joined_layer, output_layer

Hex to huc8

In [61]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='WBDHU8')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 26.6 s
Wall time: 26.7 s


In [62]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 12.9 s
Wall time: 12.9 s


In [63]:
%%time
output_layer.to_file('h'+hexLevel+'_WBDHU8.gdb',driver='OpenFileGDB')

CPU times: total: 3.92 s
Wall time: 3.92 s


In [64]:
del joined_layer, output_layer

Hex to census

In [65]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='dtl_cnty_Census_ESRI')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 9.58 s
Wall time: 9.58 s


In [66]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 14.9 s
Wall time: 14.9 s


In [67]:
%%time
output_layer.to_file('h'+hexLevel+'_Census.gdb',driver='OpenFileGDB')

CPU times: total: 3.98 s
Wall time: 3.98 s


In [68]:
del joined_layer, output_layer

Hex to huc12

In [69]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='WBDHU12')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 8.92 s
Wall time: 8.94 s


In [70]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 2.89 s
Wall time: 2.9 s


In [71]:
%%time
output_layer.to_file('h'+hexLevel+'_WBDHU12.gdb',driver='OpenFileGDB')

CPU times: total: 4.94 s
Wall time: 4.97 s


In [72]:
del joined_layer, output_layer

Hex to all at once

In [73]:
%%time
layers_to_join = ['tj_2021_us_st_cnt', 'Estuarine_Drainage_Areas', 'WBDHU8', 'dtl_cnty_Census_ESRI', 'WBDHU12']
layer_gdfs = {layer: gpd.read_file(inputs_gdb, layer=layer) for layer in layers_to_join}

CPU times: total: 47.5 s
Wall time: 47.6 s


In [74]:
%%time
for name, gdf in layer_gdfs.items():
    # Ensure the CRS is the same between the base layer and the current layer
    if not gdf.crs == base_layer.crs:
        gdf = gdf.to_crs(base_layer.crs)
    if 'Shape_Length' in gdf.columns:
        gdf.drop('Shape_Length', axis=1, inplace=True)
    if 'Shape_Area' in gdf.columns:
        gdf.drop('Shape_Area', axis=1, inplace=True)

    # Perform spatial join (inner join by default)
    base_layer = gpd.sjoin(base_layer, gdf, how="left")
    
    # Drop the `index_right` column if it exists
    if 'index_right' in base_layer.columns:
        base_layer.drop('index_right', axis=1, inplace=True)

CPU times: total: 44 s
Wall time: 44 s


In [75]:
%%time
base_layer.to_file('h'+hexLevel+'_allAtOnce.gdb',driver='OpenFileGDB')

CPU times: total: 10.3 s
Wall time: 10.4 s


In [76]:
del layers_to_join, layer_gdfs, gdf, base_layer

# HEX 8

Hex imports

In [77]:
%%time
# Define the path to the geodatabase
inputs_gdb = r'C://Research/Grid_effort/H3Grid_Inputs.gdb'

hexLevel = '8'

# Read in the H3_5 polygon grid layer as the base layer
base_layer = gpd.read_file(inputs_gdb, layer='H3_'+hexLevel)
base_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)

CPU times: total: 32.9 s
Wall time: 33 s


Hex to counties

In [78]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='tj_2021_us_st_cnt')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 1.44 s
Wall time: 1.43 s


In [79]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 14.3 s
Wall time: 14.3 s


In [80]:
%%time
output_layer.to_file('h'+hexLevel+'_tj2021.gdb',driver='OpenFileGDB')

CPU times: total: 29.6 s
Wall time: 29.7 s


In [81]:
del joined_layer, output_layer

Hex to estuarine

In [82]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='Estuarine_Drainage_Areas')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 359 ms
Wall time: 351 ms


In [83]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 10.6 s
Wall time: 10.6 s


In [84]:
%%time
output_layer.to_file('h'+hexLevel+'_Estuarine.gdb',driver='OpenFileGDB')

CPU times: total: 26.2 s
Wall time: 26.3 s


In [85]:
del joined_layer, output_layer

Hex to huc8

In [86]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='WBDHU8')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 30 s
Wall time: 30 s


In [87]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 1min 30s
Wall time: 1min 31s


In [88]:
%%time
output_layer.to_file('h'+hexLevel+'_WBDHU8.gdb',driver='OpenFileGDB')

CPU times: total: 26 s
Wall time: 26.1 s


In [89]:
del joined_layer, output_layer

Hex to census

In [90]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='dtl_cnty_Census_ESRI')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 10.3 s
Wall time: 10.3 s


In [91]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 1min 42s
Wall time: 1min 42s


In [92]:
%%time
output_layer.to_file('h'+hexLevel+'_Census.gdb',driver='OpenFileGDB')

CPU times: total: 26.5 s
Wall time: 26.6 s


In [93]:
del joined_layer, output_layer

Hex to huc12

In [94]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='WBDHU12')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 8.88 s
Wall time: 8.88 s


In [95]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 17.4 s
Wall time: 17.4 s


In [96]:
%%time
output_layer.to_file('h'+hexLevel+'_WBDHU12.gdb',driver='OpenFileGDB')

CPU times: total: 28.8 s
Wall time: 28.8 s


In [97]:
del joined_layer, output_layer

Hex to all at once

In [98]:
%%time
layers_to_join = ['tj_2021_us_st_cnt', 'Estuarine_Drainage_Areas', 'WBDHU8', 'dtl_cnty_Census_ESRI', 'WBDHU12']
layer_gdfs = {layer: gpd.read_file(inputs_gdb, layer=layer) for layer in layers_to_join}

CPU times: total: 51.3 s
Wall time: 51.3 s


In [99]:
%%time
for name, gdf in layer_gdfs.items():
    # Ensure the CRS is the same between the base layer and the current layer
    if not gdf.crs == base_layer.crs:
        gdf = gdf.to_crs(base_layer.crs)
    if 'Shape_Length' in gdf.columns:
        gdf.drop('Shape_Length', axis=1, inplace=True)
    if 'Shape_Area' in gdf.columns:
        gdf.drop('Shape_Area', axis=1, inplace=True)

    # Perform spatial join (inner join by default)
    base_layer = gpd.sjoin(base_layer, gdf, how="left")
    
    # Drop the `index_right` column if it exists
    if 'index_right' in base_layer.columns:
        base_layer.drop('index_right', axis=1, inplace=True)

CPU times: total: 4min 14s
Wall time: 4min 14s


In [100]:
%%time
base_layer.to_file('h'+hexLevel+'_allAtOnce.gdb',driver='OpenFileGDB')

CPU times: total: 46.1 s
Wall time: 46.3 s


In [101]:
del layers_to_join, layer_gdfs, gdf, base_layer

# HEX 9

Hex imports

In [102]:
%%time
# Define the path to the geodatabase
inputs_gdb = r'C://Research/Grid_effort/H3Grid_Inputs.gdb'

hexLevel = '9'

# Read in the H3_5 polygon grid layer as the base layer
base_layer = gpd.read_file(inputs_gdb, layer='H3_'+hexLevel)
base_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)

CPU times: total: 3min 50s
Wall time: 3min 50s


Hex to counties

In [103]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='tj_2021_us_st_cnt')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 2.08 s
Wall time: 2.08 s


In [104]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 1min 48s
Wall time: 1min 48s


In [105]:
%%time
output_layer.to_file('h'+hexLevel+'_tj2021.gdb',driver='OpenFileGDB')

CPU times: total: 3min 22s
Wall time: 3min 22s


In [106]:
del joined_layer, output_layer

Hex to estuarine

In [107]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='Estuarine_Drainage_Areas')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 188 ms
Wall time: 195 ms


In [108]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 1min 45s
Wall time: 1min 45s


In [109]:
%%time
output_layer.to_file('h'+hexLevel+'_Estuarine.gdb',driver='OpenFileGDB')

CPU times: total: 3min 1s
Wall time: 3min 1s


In [110]:
del joined_layer, output_layer

Hex to huc8

In [111]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='WBDHU8')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 31.9 s
Wall time: 32 s


In [112]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 12min 25s
Wall time: 12min 25s


In [113]:
%%time
output_layer.to_file('h'+hexLevel+'_WBDHU8.gdb',driver='OpenFileGDB')

CPU times: total: 3min
Wall time: 3min 1s


In [114]:
del joined_layer, output_layer

Hex to census

In [115]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='dtl_cnty_Census_ESRI')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 10 s
Wall time: 10 s


In [116]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 12min 13s
Wall time: 12min 13s


In [117]:
%%time
output_layer.to_file('h'+hexLevel+'_Census.gdb',driver='OpenFileGDB')

CPU times: total: 3min 6s
Wall time: 3min 6s


In [118]:
del joined_layer, output_layer

Hex to huc12

In [119]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='WBDHU12')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 8.75 s
Wall time: 8.75 s


In [120]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 1min 57s
Wall time: 1min 57s


In [121]:
%%time
output_layer.to_file('h'+hexLevel+'_WBDHU12.gdb',driver='OpenFileGDB')

CPU times: total: 3min 10s
Wall time: 3min 10s


In [122]:
del joined_layer, output_layer

Hex to all at once

In [123]:
%%time
layers_to_join = ['tj_2021_us_st_cnt', 'Estuarine_Drainage_Areas', 'WBDHU8', 'dtl_cnty_Census_ESRI', 'WBDHU12']
layer_gdfs = {layer: gpd.read_file(inputs_gdb, layer=layer) for layer in layers_to_join}

CPU times: total: 53.7 s
Wall time: 53.8 s


In [ ]:
%%time
for name, gdf in layer_gdfs.items():
    # Ensure the CRS is the same between the base layer and the current layer
    if not gdf.crs == base_layer.crs:
        gdf = gdf.to_crs(base_layer.crs)
    if 'Shape_Length' in gdf.columns:
        gdf.drop('Shape_Length', axis=1, inplace=True)
    if 'Shape_Area' in gdf.columns:
        gdf.drop('Shape_Area', axis=1, inplace=True)

    # Perform spatial join (inner join by default)
    base_layer = gpd.sjoin(base_layer, gdf, how="left")
    
    # Drop the `index_right` column if it exists
    if 'index_right' in base_layer.columns:
        base_layer.drop('index_right', axis=1, inplace=True)

In [ ]:
%%time
base_layer.to_file('h'+hexLevel+'_allAtOnce.gdb',driver='OpenFileGDB')

In [ ]:
del layers_to_join, layer_gdfs, gdf, base_layer